# neighborhood-based collaborative filtering

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [14]:
class uuCF(object):
    def __init__(self, Y_data, k, sim_func=cosine_similarity):
        # a 2d array of shape (n_users, 3)
        # each row of Y_data has form [user_id, item_id, rating]
        self.Y_data = Y_data
        # number of neighborhood
        self.k = k
        # similarity function, default: cosine_similarity
        self.sim_func = sim_func
        # normalize data
        self.Ybar = None
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1  # number of users
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1  # number of items

    def fit(self):
        users = self.Y_data[:, 0]  # all users - first col of the Y_data
        self.Ybar = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))

        for n in range(self.n_users):
            # row indices of ratings made by user n
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all items rated by user n
            item_ids = self.Y_data[ids, 1]
            ratings = self.Y_data[ids, 2]  # ratings made by user n
            # avoid zero division
            self.mu[n] = np.mean(ratings) if ids.size > 0 else 0
            self.Ybar[ids, 2] = ratings - self.mu[n]

        # form the rating matrix as a sparse matrix.
        self.Ybar = sparse.coo_matrix(
            (self.Ybar[:, 2], (self.Ybar[:, 1], self.Ybar[:, 0])), (self.n_items, self.n_users)).tocsr()
        self.S = self.sim_func(self.Ybar.T, self.Ybar.T)

    def pred(self, u, i):
        """ predict the rating of user u for item i"""
        # find item i
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        # all users who rated i
        users_rated_i = self.Y_data[ids, 0].astype(np.int32)
        # similarity of u and users who rated i
        sim = self.S[u, users_rated_i]

        # most k similar users
        nns = np.argsort(sim)[-self.k:]
        # and the corresponding similarities
        nearest_s = sim[nns]
        # the corresponding ratings
        r = self.Ybar[i, users_rated_i[nns]]
        # a small number to avoid zero division
        eps = 1e-8
        return (r*nearest_s).sum() / (np.abs(nearest_s).sum() + eps) + self.mu[u]


## MovieLens

In [15]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols)

rate_train = ratings_base.to_numpy()
rate_test = ratings_test.to_numpy()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [16]:
rs = uuCF(rate_train, k = 40)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in range(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print('User-user CF, RMSE =', RMSE)

User-user CF, RMSE = 0.9766140289287265


# item-item cf

In [17]:
rate_train = rate_train[:, [1, 0, 2]]
rate_test  = rate_test[:, [1, 0, 2]]

rs = uuCF(rate_train, k = 40)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in range(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print('Item-item CF, RMSE =', RMSE)

Item-item CF, RMSE = 0.9688460838682366
